# 对表层，100m层的S,U,V,T,E-P进行EOF分解

In [4]:
import numpy as np
import pandas as pd 
import xarray as xr
import xeofs as xe
from scipy.signal import detrend
from scipy.ndimage import gaussian_filter
# from eofs.standard import Eof
# from pykalman import KalmanFilter
np.set_printoptions(suppress=True)
# %matplotlib inline
%matplotlib qt
%pwd

'c:\\Users\\jupyter\\indian_ocean\\whole_indian_ocean\\reconstruct_data'

## 导入数据， 去除线性趋势，并标准化

In [ ]:
#定义一个去掉线性趋势的函数
def detrend_skip_nan(ts):
    ts = ts.copy()  # 创建可写副本
    valid = ~np.isnan(ts)
    if np.sum(valid) < 2:
        return ts  #nan值太多，直接返回原始数据(这里是陆地的全nan值)
    ts_valid = ts[valid]
    ts_detrend = detrend(ts_valid, axis=0)  #数据不存在nan值的时候，才用detrend函数
    ts[valid] = ts_detrend
    return ts

def xr_detrend(da):
    '''去除数据中的线性趋势'''
    da = da.transpose('time', 'lon', 'lat')
    
    detrended = xr.apply_ufunc(
        detrend_skip_nan,  #这里apply_ufunc传递的是time*空间点的数据，遍历每个空间点的时间序列
        da,        #当遇到全是nan值的点
        input_core_dims=[['time']],
        output_core_dims=[['time']],
        vectorize=True,
       ) 
    return detrended

surface_data = xr.open_dataset(r'D:\all_processed_data\surface_mean&clim&detrended_data\surface_data.nc')
hundred_data = xr.open_dataset(r'D:\all_processed_data\hundred_mean&clim&detrended_data\hundred_data.nc')


surface_S = surface_data['detrended_surface_S']
surface_U = surface_data['detrended_surface_U']
surface_V = surface_data['detrended_surface_V']
surface_T = surface_data['detrended_surface_T']
surface_EP = surface_data['detrended_surface_EP']

hundred_S = hundred_data['detrended_100m_S']
hundred_U = hundred_data['detrended_100m_U']
hundred_V = hundred_data['detrended_100m_V']
hundred_T = hundred_data['detrended_100m_T']

## 去除线性趋势
surface_S = xr_detrend(surface_S)
surface_U = xr_detrend(surface_U)
surface_V = xr_detrend(surface_V)
surface_T = xr_detrend(surface_T)
surface_EP = xr_detrend(surface_EP)

hundred_S = xr_detrend(hundred_S)
hundred_U = xr_detrend(hundred_U)
hundred_V = xr_detrend(hundred_V)
hundred_T = xr_detrend(hundred_T)

## 标准化
surface_S = (surface_S - surface_S.mean(dim='time')) / surface_S.std(dim='time')
surface_U = (surface_U - surface_U.mean(dim='time')) / surface_U.std(dim='time')
surface_V = (surface_V - surface_V.mean(dim='time')) / surface_V.std(dim='time')
surface_T = (surface_T - surface_T.mean(dim='time')) / surface_T.std(dim='time')
surface_EP = (surface_EP - surface_EP.mean(dim='time')) / surface_EP.std(dim='time')

hundred_S = (hundred_S - hundred_S.mean(dim='time')) / hundred_S.std(dim='time')
hundred_U = (hundred_U - hundred_U.mean(dim='time')) / hundred_U.std(dim='time')
hundred_V = (hundred_V - hundred_V.mean(dim='time')) / hundred_V.std(dim='time')
hundred_T = (hundred_T - hundred_T.mean(dim='time')) / hundred_T.std(dim='time')

data_var_dict = dict(surface_S=surface_S, surface_U=surface_U, surface_V=surface_V,
                     surface_T=surface_T, surface_EP=surface_EP,
                     hundred_S=hundred_S, hundred_U=hundred_U, hundred_V=hundred_V, hundred_T=hundred_T)
description_dict = dict(description='去除线性趋势并标准化后的去气候态数据， 用以进行EOF分解')
dataset = xr.Dataset(data_vars=data_var_dict, attrs=description_dict)

dataset.to_netcdf(r'D:\all_processed_data\standardized_detrended_surface&hundred_SUVTEP\for_EOF.nc')

In [4]:
dataset

<xarray.Dataset> Size: 8GB
Dimensions:     (lon: 841, lat: 574, time: 240)
Coordinates:
  * lon         (lon) float64 7kB 40.0 40.08 40.17 40.25 ... 109.8 109.9 110.0
  * lat         (lat) float64 5kB -14.99 -14.91 -14.83 ... 30.81 30.89 30.96
  * time        (time) datetime64[ns] 2kB 2001-01-01 2001-02-01 ... 2020-12-01
Data variables:
    surface_S   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    surface_U   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    surface_V   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    surface_T   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    surface_EP  (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    hundred_S   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    hundred_U   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    hundred_V   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
    hundred_T   (lon, lat, time) float64 927MB nan nan nan nan ... nan nan nan
Attributes:
    description:  去除线性趋势并标准化后的去气候态数据， 用以进行EOF分解

## 对S, U, V, T, E-P进行分解

### 对表层的S,T,E-P和100m的S,T进行EOF分解

In [3]:
data = xr.open_dataset(r'D:\all_processed_data\standardized_detrended_surface&hundred_SUVTEP\for_EOF.nc')
##这几个数据的一些点在某些时间上为nan值，无法进行EOF分解，因此我们索性将他们都设置为nan值
surface_EP = data['surface_EP']
surface_EP_mask = surface_EP.isnull().any(dim='time')
surface_EP = surface_EP.where(~surface_EP_mask)

hundred_S = data['hundred_S']
hundred_S_mask = hundred_S.isnull().any(dim='time')
hundred_S = hundred_S.where(~hundred_S_mask)

hundred_T = data['hundred_T']
hundred_T_mask = hundred_T.isnull().any(dim='time')
hundred_T = hundred_T.where(~hundred_T_mask)

variable_dict = dict(
    surface_S=data['surface_S'], surface_T=data['surface_T'], surface_EP=surface_EP,
    hundred_S=hundred_S, hundred_T=hundred_T)

eof_variable_list = []
save_variable_list = [[i+'_eofs', i+'_pcs', i+'_vars']    for i in variable_dict.keys()]
save_variable_list = [i for sublist in save_variable_list for i in sublist]
def apply_EOF(da):
    model = xe.single.EOF(n_modes=5, use_coslat=True)
    model.fit(da, dim='time')
    eofs = model.components()
    pcs = model.scores()
    vars = model.explained_variance_ratio()
    # return xr.Dataset(dict(eofs=eofs, pcs=pcs, vars=vars), attrs=dict(description='EOF分解的结果'))
    return eofs, pcs, vars
# eofs, pcs, vars= apply_EOF(variable_dict['surface_S'])

for variable in variable_dict.keys():
    eofs, pcs, vars = apply_EOF(variable_dict[variable])
    eof_variable_list.append(eofs)
    eof_variable_list.append(pcs)
    eof_variable_list.append(vars)

dataset_variable_dict={save_variable_list[i]:eof_variable_list[i] for i in range(15)}
eof_data = xr.Dataset(dataset_variable_dict, attrs=dict(description='表层的S,T,E-P\n100m层的S,T进行EOF分解的结果'))
# 遍历所有变量，删除包含无效类型的属性(使用eof分解后，一些过程数据可能会存到solver_kwargs中，需要删掉，否则无法保存)
for var in eof_data.data_vars:
    if 'solver_kwargs' in eof_data[var].attrs:
        del eof_data[var].attrs['solver_kwargs']

# 如果全局属性中存在，也删除
if 'solver_kwargs' in eof_data.attrs:
    del eof_data.attrs['solver_kwargs']

eof_data.to_netcdf(r'D:\all_processed_data\EOF_data\surface_hundred_STEP_EOF.nc')

### 对表层和100m层的U,V进行联合complex EOF分解

#### 导入并处理数据
数据噪音太大，对数据进行时空高斯滤波，以提高eof模态的方差贡献度

In [47]:
def apply_gaussian_filter(data_array, temporal_sigma=6, spatial_sigma=3):
    data_array = data_array.transpose('time', 'lon', 'lat')
    return xr.apply_ufunc(
        lambda data: gaussian_filter(data, sigma=(temporal_sigma, spatial_sigma, spatial_sigma)),  # 时空维度滤波
        data_array,
        input_core_dims=[['time', 'lon', 'lat']],
        output_core_dims=[['time', 'lon', 'lat']],
        vectorize=True)

data = xr.open_dataset(r'D:\all_processed_data\standardized_detrended_surface&hundred_SUVTEP\for_EOF.nc')
# 标准化
surface_U = (data['surface_U'] - data['surface_U'].mean(dim='time')) / data['surface_U'].std(dim='time')
surface_V = (data['surface_V'] - data['surface_V'].mean(dim='time')) / data['surface_V'].std(dim='time')
hundred_U = (data['hundred_U'] - data['hundred_U'].mean(dim='time')) / data['hundred_U'].std(dim='time')
hundred_V = (data['hundred_V'] - data['hundred_V'].mean(dim='time')) / data['hundred_V'].std(dim='time')

#构造复数矩阵
surface_UV = surface_U + 1j* surface_V
hundred_UV = hundred_U + 1j* hundred_V

#时空高斯滤波
filtered_surface_UV = apply_gaussian_filter(surface_UV)
filtered_hundred_UV = apply_gaussian_filter(hundred_UV)

### 进行 complex EOF分解

In [49]:
def apply_CEOF(da):
    model = xe.single.ComplexEOF(n_modes=5, use_coslat=True, random_state=7)
    model.fit(da, dim='time')
    spatial_amp = model.components_amplitude()
    spatial_phase = model.components_phase()
    eofs = model.components()
    pcs = model.scores()
    vars = model.explained_variance_ratio()
    return [spatial_amp, spatial_phase, eofs, pcs, vars]

surface_CEOF_data = apply_CEOF(filtered_surface_UV)
hundred_CEOF_data = apply_CEOF(filtered_hundred_UV)

surface_UV_CEOF_data = xr.Dataset(dict(surface_UV_amp=surface_CEOF_data[0], surface_UV_phase=surface_CEOF_data[1],
                surface_UV_eofs_real=surface_CEOF_data[2].real, surface_UV_eofs_imag=surface_CEOF_data[2].imag,
                surface_UV_pcs_real=surface_CEOF_data[3].real, surface_UV_pcs_imag=surface_CEOF_data[3].imag,
                surface_UV_vars=surface_CEOF_data[4]), attrs=dict(description='海表速度的CEOF分解结果'))

hundred_UV_CEOF_data = xr.Dataset(dict(hundred_UV_amp=hundred_CEOF_data[0], hundred_UV_phase=hundred_CEOF_data[1],
                hundred_UV_eofs_real=hundred_CEOF_data[2].real, hundred_UV_eofs_imag=hundred_CEOF_data[2].imag,
                hundred_UV_pcs_real=hundred_CEOF_data[3].real, hundred_UV_pcs_imag=hundred_CEOF_data[3].imag,
                hundred_UV_vars=hundred_CEOF_data[4]), attrs=dict(description='100m速度的CEOF分解结果'))

for var in surface_UV_CEOF_data:
    if 'solver_kwargs' in surface_UV_CEOF_data[var].attrs:
        del surface_UV_CEOF_data[var].attrs['solver_kwargs']

# 如果全局属性中存在，也删除
if 'solver_kwargs' in surface_UV_CEOF_data.attrs:
    del surface_UV_CEOF_data.attrs['solver_kwargs']

for var in hundred_UV_CEOF_data:
    if 'solver_kwargs' in hundred_UV_CEOF_data[var].attrs:
        del hundred_UV_CEOF_data[var].attrs['solver_kwargs']

# 如果全局属性中存在，也删除
if 'solver_kwargs' in hundred_UV_CEOF_data.attrs:
    del hundred_UV_CEOF_data.attrs['solver_kwargs']

surface_UV_CEOF_data.to_netcdf(r'D:\all_processed_data\EOF_data\surface_UV_CEOF_data.nc')
hundred_UV_CEOF_data.to_netcdf(r'D:\all_processed_data\EOF_data\hundred_UV_CEOF_data.nc')

c:\Users\Admin\anaconda3\envs\xeofs\Lib\site-packages\scipy\sparse\linalg\_eigen\_svds.py:477: UserWarning: Exited at iteration 20 with accuracies 
[0.00000003 0.00000014 0.00000014 0.00002582 0.03978017]
not reaching the requested tolerance 3.5762786865234375e-06.
Use iteration 21 instead with accuracy 
0.007961260432936068.

  _, eigvec = lobpcg(XH_X, X, tol=tol ** 2, maxiter=maxiter,
c:\Users\Admin\anaconda3\envs\xeofs\Lib\site-packages\scipy\sparse\linalg\_eigen\_svds.py:477: UserWarning: Exited postprocessing with accuracies 
[0.00000003 0.00000014 0.00000014 0.00002582 0.03978017]
not reaching the requested tolerance 3.5762786865234375e-06.
  _, eigvec = lobpcg(XH_X, X, tol=tol ** 2, maxiter=maxiter,
c:\Users\Admin\anaconda3\envs\xeofs\Lib\site-packages\scipy\sparse\linalg\_eigen\_svds.py:477: UserWarning: Exited at iteration 20 with accuracies 
[0.00000003 0.00000009 0.00000033 0.00001062 0.03993944]
not reaching the requested tolerance 3.5762786865234375e-06.
Use iteration 21 i